# CA1: Curve Fitting Problem

In the first computer assignment we have been asked to design a genetic algorithm so that it converges on a solution to the "Curve Fitting Problem". Curve Fitting's goal is finding a polynomial representation for a set of given points, in a way that the polynomial representation corresponds each point.

For the sake of simplicity, in this project, the degree of the result is given as a parameter to the program, as well as the interval each coefficient must be in. Also, it is considered that points in the given sample must be in $\mathbb{Z}^2$ and each coefficient in the polynomial representation must be in $\mathbb{Z}$.

## Section 0: Formulation of the Problem

We formulate the problem by this interpretation:

- the size of the sample is $n$,
- each point in the sample is shown by a 2-tuple $(x_i, y_i)$,
- the given interval for coefficents is $[c_{min}, c_{max})$,
- the degree of the target polynomial is $d$.

Also the polynomials are shown as $p_i(x)$ while the optimal answer is represented as $p^*(x)$.

In [1]:
chromosome_length = int(input("polynomial degree: ")) + 1
gene_interval = tuple(map(int, input("coefficients' interval: ").split()))
sample_size = int(input("number of samples: "))
sample = []
for i in range(sample_size):
    point = tuple(
            map(int, input(f"point {i + 1}: ").split())
            )
    sample.append(point)

## Section 1: Demonstrating Basic Ideas

Implementation of a genetic algorithm is reliant on how we define each required concept. To do so, the approach taken was defining them with no excessive complication. In this approach, a **Gene** is defined as a coefficient in a polynomial representation. Thus, a gene basically is an integer. Genes are referred to with letter $g$. Other conceptual entities are described as their basic definition in genetic algorithm design:

- A **chromosome** is a sequence of genes. A chromosome is referred to as a $(d+1)$-tuple.  
$ch_i=(g_0, g_1, \dots, g_d)$.
- Each **cycle** includes a sequence of actions performed on a population to create a new population.
- **Population** is a sequence of chromosomes corresponding. The size of population is referred to by $m$.
- **Evaluation function** is the score of each chromosome. It must be increasing as chromosomes get closer to the goal.  
$evaluation\_function: chromosomes \rightarrow \mathbb{R}$
- **Selection** is an action taken to choose members of the population that will survive to the next cycle.  
$select: chromosomes^m \rightarrow chromosomes^m$
- **Crossover function** is a function that creates two new chromosomes out of two parent chromosomes.  
$crossover: chromosomes^2 \rightarrow chromosomes^2$
- **Mutation function** is a function that mutates a gene to another gene.  
$mutate: genes \rightarrow genes$
- **Crossover probability** is the probability by which crossover function is called for two selected chromosomes.  
$Pr_{crossover}$
- **Mutation probability** is the probability by which a gene is mutated.  
$Pr_{mutation}$

In our design it was determined that a number of good chromosomes would not get mutated at all. This number is referred to by $N_{not\_mutated}$.

To implement all these factors eligibly, they are reserved in a `Config` class. In this class two more methods are implemented that are not mentioned:
- `get_probability` which returns a sequence of probabilities, each corresponding to the probability a chromosome could get selected by, with respect to the order of chromosomes in the population,
- and `is_absolute_solution_found` that returns true when there is chromosome in the population that has maximum evaluation possible.

In [2]:
from genetic_utils import Gene, Chromosome
from typing import List
from random import uniform

class Config:
    population_length = int(input("population_length: "))
    mutation_probability = float(input("mutation_probability: "))
    not_mutated_count = int(input("not_mutated_count: "))
    crossover_probability = float(input("crossover_probability: "))
    def mutation_function(gene: Gene) -> Gene:
        return Gene(int(uniform(*gene_interval)))
    
    def crossover_function(
            chromosome1: Chromosome, 
            chromosome2: Chromosome) -> [Chromosome, Chromosome]:
        gene_sequence1 = []
        gene_sequence2 = []
        choice_vector = [chromosome1, chromosome2]
        for i in range(chromosome_length):
            if i == int(chromosome_length / 2):
                choice_vector = choice_vector[::-1]
            gene_sequence1.append(choice_vector[0].get_gene(i))
            gene_sequence2.append(choice_vector[1].get_gene(i))

        new_chromosome1 = Chromosome(gene_sequence1)
        new_chromosome2 = Chromosome(gene_sequence2)
        return [new_chromosome1, new_chromosome2]
    
    def evaluation_function(chromosome: Chromosome) -> float:
        calculated_value = 0
        for point in sample:
            x, y = point
            f_x = 0
            for i in range(chromosome_length):
                gene = chromosome.get_gene(i)
                f_x += gene.value * (x ** i)
            calculated_value -= abs(y - f_x)
        return calculated_value
    
    def get_probabilities(population: List[Chromosome]) -> List[float]:
        values = [Config.evaluation_function(chromosome) \
            for chromosome in population]
        value_times_probabilities = 1 / sum([1 / x for x in values])
        probabilities = [value_times_probabilities / x for x in values]
        return probabilities
    
    def is_absolute_solution_found(population: List[Chromosome]) -> bool:
        for chromosome in population:
            if Config.evaluation_function(chromosome) == 0:
                return True
        return False

## Section 2: Instantiating Genetic Algorithm

In order to pledge oneself to the concepts, a considerably general implementation of genetic algorithms was written in a package called `genetic_utils`. In this package there are three entities:

- an implementation of a `Gene` class,
- an implementation of a `Chromosome` class,
- and an implementation of a `GeneticAlgorithm` class that has following methods:
  - `do_cycle` that performs a cycle of `do_selection`, `do_crossover`, and `do_mutation`,
  - `generate_population` that randomly creates $m$ chromosomes which fulfills the conditions,
  - and `show_population` and `get_best`, that in each state the algorithm is they can be used to print all the members of the population and to print the best member in the population regarding $evaluation\_function$, respectively.

Thus, we store this instance of genetic algorithm which is described in this section, in a variable named `algorithm`.

In [3]:
from genetic_utils import GeneticAlgorithm

algorithm = GeneticAlgorithm(
        gene_interval,
        chromosome_length,
        Config.population_length,
        Config.mutation_probability,
        Config.mutation_function,
        Config.not_mutated_count,
        Config.crossover_probability,
        Config.crossover_function,
        Config.evaluation_function,
        Config.get_probabilities,
        Config.is_absolute_solution_found)

## Section 3: Generating The First Population

To do so, we simply can call `generate_population` method for the `algorithm`.

In [9]:
algorithm.generate_population()
algorithm.show_population()

show_population: population contains:
index=0: chromosome=(3, 3, 3)
index=1: chromosome=(1, 9, 4)
index=2: chromosome=(6, 3, 7)
index=3: chromosome=(4, 0, 2)
index=4: chromosome=(4, 7, 2)
index=5: chromosome=(2, 4, 9)
index=6: chromosome=(4, 0, 0)
index=7: chromosome=(3, 8, 8)
index=8: chromosome=(0, 6, 9)
index=9: chromosome=(5, 5, 6)
--------End--------


## Section 4: Running the Algorithm for a Given Number of Cycles

To do so, in the `GeneticAlgorithm`, method `do_cycles` is put to run `do_cycle` for multiple times so long as it has not reached to an *absolute solution*.  
The number of cycles is referred to by $N_{cycles}$.

In [10]:
cycle_count = int(input("number of cycles: "))
algorithm.do_cycles(cycle_count, do_print=False)
print(f"the best chromosome so far: {algorithm.get_best()}")
algorithm.show_population()

the best chromosome so far: (1, 1, 1)
show_population: population contains:
index=0: chromosome=(1, 1, 1)
index=1: chromosome=(2, 1, 1)
index=2: chromosome=(0, 1, 1)
index=3: chromosome=(0, 8, 1)
index=4: chromosome=(2, 0, 1)
index=5: chromosome=(2, 0, 1)
index=6: chromosome=(2, 0, 1)
index=7: chromosome=(4, 1, 1)
index=8: chromosome=(0, 0, 1)
index=9: chromosome=(0, 0, 1)
--------End--------


## Section 5: Time Complexity

- `evaluation_function`: $O(nd)$
- `get_probabilities`: $O(m \times complexity($`evaluation_function`$)) \rightarrow O(mnd)$
- `do_selection`: $O(complexity($`get_probabilities`$) + m^2) \rightarrow O(mnd + m^2)$ 
- `do_crossover`: $O(md)$
- `do_mutation`: $O(md)$
- `is_absolute_solution_found`: $O(m \times complexity($`evaluation_function`$)) \rightarrow O(mnd)$
- `do_cycle`:  
$O(complexity($`is_absolute_solution_found`$) + complexity($`do_selection`$) + complexity($`crossover`$) + complexity($`do_mutation`$)) \rightarrow O(mnd + m^2)$
- `do_cycles`: $O(N_{cycles} \times complexity($`do_cycle`$)) \rightarrow O(N_{cycles}m(nd + m))$

$$
Total\space Complexity = O(N_{cycles}m(nd + m))
$$

## Section 6: Space Complexity

Since at each instant, there are only $m$ chromosomes stored in the memory:
$$ Total\space Complexity = O(md + n)$$

## Section 7: Questions

* **Question #1: What consequences does it have when the population size is too small or too big?**  
If $m$ is too small, it is going to take a myriad of cycles to find the solution. It is also more likely to get stuck in local maxima.   
On the other hand, if $m$ is too big, time usage is going to increase drastically for time complexity being $O(N_{cycles}(nd + m)m)$.

* **Question #2: If the size of the population increases in each cycle, what impacts it is going to have on the precision and the speed of the algorithm?**  
As algorithm goes forward, the convergence of chromosomes shapes toward a local solution. Therefore, as cycles pass, the size of needed chromosomes decreases since it is getting closer to the convergence. Now, if we decide to increase population in each cycle, it is going to result in a increase in time complexity while adding statistically no precision to our algorithm.

* **Question #3: What are the impacts of mutation and crossover operations? Can any of them be removed? Why?**  
Crossover function enables a genetic algorithm to create new chromosomes out of a set of *"good"* parents in a way that they inherit their *"good"* qualities.  
Nevertheless, mutation function enables a genetic algorithm to create completely new chromosomes that may not be previously seen in any cycle's population.  
If crossovering were not included, in each cycle, the algorithm would not create new chromosomes that necessarily keep previous qualities. Instead, it would create new chromosomes randomly out of old ones without considering the evaluation measure.  
On the opposite side, if mutation were not included, the algorithm would get stuck in local maxima. In other words, only chromosomes that were in the range of the first population could be seen in the any cycle's population.

* **Question #4: In your opinion, what can be done to enable the algorithm to find solutions faster?**  
In this particular instance, because the domain of $p(x)$ is $\mathbb{Z}$, for chromosome $ch=(g_0, g_1, \dots, g_d)$ we know the impact of $g_i$ is more than the impact of $g_{i-1}$ for $i=1,2,\dots,d$. Therefore, it could be a good idea to implement this factor in $Pr_{mutation}$ and also in the crossover function. For instance, if the the evaluation function is less than a particular number, then crossovering should happen from a gene with more impact.

* **Question #5: Despite all the effort, it is still possible for chromosomes to stay unchanged after some cycles. Why is it possible and what problems it will cause? What suggestions do you have to resolve this issue?**  
If the population is stuck in a local maximum and $Pr_{mutation}$ is not high enough to change the direction of the population very fast, the aforementioned circumstance will appear. The result of this incident is simply the fact that reaching the goal chromosome would *seem impossible*. Based on the reason mutation is defined in genetic algorithms, to resolve this issue, it is suggested to increase $Pr_{mutation}$ to a value that gives more chance to *exploration* of new chromosomes. Also, with bigger or more diverse sample size, the chance of getting into local maxima would decrease since their corresponding chromosomes would be fewer.

* **Question #6: What do you suggest for algorithm finishing due to the assumption that the problem does not have a solution?**  
In my implementation, there exists a function called `is_absolute_solution_found`. One way to finish the algorithm in these cases by changing this function to being true when the evaluation is *good enough* rather than being the maximum value possible.  
Another approach is keeping a value that corresponds how much the population got changed during the last cycle. If this value converged on a number after some cycles, then the algorithm would terminate itself.

* **Question #7: When the degree of the polynomial increases, how it affects time complexity?**  
As it was mentioned in Section 5, the time complexity of this algorithm is $O(N_{cycles}m(nd+m))$. Therefore, by the increase in $d$, the time complexity would increase linearly.

* **Question #8: In your opinion, how enlarging or reducing the size of sample can have impact on algorithm's execuation?**  
A large sample size would result in a more precise evaluation measure, which literally causes better evaluation of each chromosome. Also, a more precise evaluation measure, can belittle the chance that the algorithm gets stuck in a local maximum. However, the trade-off is by increasing the size of sample, the time complexity of `evaluation_function` will increase.

## Conclusion

Hitherto the class `GeneticAlgorithm` was designed to satisfy any problem that could be implemented as a genetic algorithm. Also, an instantiation of it was run to approximately solve *Curve Fitting Problem*. On this way, issues such as time and space complexity were discussed explicitly, alongside the answer to the eight questions attached to the assignment.  
Even though the goal of this assignment was probably just to give students an educational view to genetic algorithms, it did also provide a valuable package that could be used in the future.